In [ ]:
# # Data fetching and filtering
# data_hg = pd.read_excel("fcurves_data.xlsx", sheet_name = "HG")
# data_hg = data_hg[data_hg.iloc[:, 1:].isnull().all(axis=1) == False].reset_index(drop = True) # Keep those rows where not all values are null (drop those where all values are null)

# # Data plotting
# num_dates = data_hg.shape[0]
# num_contracts = data_hg.shape[1] - 1 # since first column is of Date
# data_hg["curve_type"] = -1
# for i in range(num_dates):
    
#     # backwardation
#     dec = True
#     for j in range(1, num_contracts):
#         if(data_hg.iloc[i, j+1]>data_hg.iloc[i, j]):
#             dec = False
#             break
#     if(dec == True):
#         data_hg.loc[i, "curve_type"] = 1 # backwardation (What it was giving earlier was Na rows)
#         continue
    
#     # contango
#     inc = True
#     for j in range(1, num_contracts):
#         if(data_hg.iloc[i, j+1]<data_hg.iloc[i, j]):
#             inc = False
#             break
#     if(inc == True):
#         data_hg.loc[i, "curve_type"] = 2 # contango
#         continue
    
#     # humped peak
#     j = 1
#     while(j<num_contracts and data_hg.iloc[i, j+1] >= data_hg.iloc[i, j]): # Increasing
#         j += 1
#     while(j<num_contracts and data_hg.iloc[i, j+1] <= data_hg.iloc[i, j]): # Decreasing
#         j += 1
#     if(j == num_contracts):
#         data_hg.loc[i, "curve_type"] = 3 # humped with peak
#         continue

#     # humped trough
#     j = 2
#     while(j<num_contracts-1 and data_hg.iloc[i, j+1] <= data_hg.iloc[i, j]): # Decreasing
#         j += 1
#     while(j<num_contracts-1 and data_hg.iloc[i, j+1] >= data_hg.iloc[i, j]): # Increasing
#         j += 1
#     if(j == num_contracts-1):
#         data_hg.loc[i, "curve_type"] = 4 # humped with trough
#         continue

#     data_hg.loc[i, "curve_type"] = 5

#### Idea 2

In [ ]:
# Data fetching and filtering
data_hg = pd.read_excel("fcurves_data.xlsx", sheet_name = "HG")
data_hg = data_hg[data_hg.iloc[:, 1:].isnull().all(axis=1) == False].reset_index(drop = True) # Keep those rows where not all values are null (drop those where all values are null)

# Data plotting
num_dates = data_hg.shape[0]
num_contracts = data_hg.shape[1] - 1 # since first column is of Date
data_hg["curve_type"] = -1
for i in range(num_dates):
    v = data_hg.iloc[i, 1: num_contracts+1]
    maxi = v.argmax() + 1
    mini = v.argmin() + 1
    lmf = abs(v[-1] - v[0])
    maxmf = abs(v[maxi-1] - v[0])
    minmf = abs(v[mini-1] - v[0])

    # Contango: maxi in last 3-4, min in first 3-4
    if(maxi >= num_contracts - 4 and mini <= 4):
        curve_type = 1
    # Backwardation: maxi in first 3-4, mini in last 3-4
    elif(maxi <= 4 and mini >= num_contracts - 4):
        curve_type = 2 
    # Humped peak: maxi in middle 8-10
    elif(num_contracts//2-5<=maxi<=num_contracts//2+5):
        curve_type = 3
    # Humped trough: mini in middle 8-10
    elif(num_contracts//2-5<=mini<=num_contracts//2+5):
        curve_type = 4
    # flat
    else:
        curve_type = 5

    data_hg.loc[i, "curve_type"] = curve_type

data_hg["Year"] = data_hg["Dates"].transform(lambda x: x.year)

In [ ]:
maturity_vec = [T for T in range(1, num_contracts+1)]
temp = data_hg[data_hg.curve_type == 5].reset_index(drop = True)
for i in range(10):
    plt.plot(maturity_vec, temp.iloc[i, 1: num_contracts+1], label = temp.iloc[i, 0].strftime("%Y-%m-%d"))
# plt.legend()
plt.show()

In [ ]:
# Plots in bunches of 100
maturity_vec = [T for T in range(1, num_contracts+1)]
for i in range(1000):
    plt.plot(maturity_vec, data_hg.iloc[i, 1:num_contracts+1], label = data_hg.iloc[i, 0].strftime("%Y-%m-%d"))
    if(i%100 == 99):
        plt.show()
plt.legend()
plt.show()

### Best Idea

In [ ]:
def determine_curve_type(v):
    lv = len(v)
    maxi = v.argmax() + 1
    mini = v.argmin() + 1
    lmf = abs(v.iloc[-1] - v.iloc[0])
    maxmf = abs(v.iloc[maxi-1] - v.iloc[0])
    minmf = abs(v.iloc[mini-1] - v.iloc[0])

    # Contango: maxi in last 3-4, min in first 3-4
    if(maxi >= lv - 4 and mini <= 4):
        curve_type = 1
    # Backwardation: maxi in first 3-4, mini in last 3-4
    elif(maxi <= 4 and lmf>=0.9*minmf):
        curve_type = 2 
    # Humped peak: maxi in middle 8-10
    elif(lv//2-5<=maxi<=lv//2+5):
        curve_type = 3
    # Humped trough: mini in middle 8-10
    elif(lv//2-5<=mini<=lv//2+5 and minmf>=3*lmf):
        curve_type = 4
    # flat
    else:
        curve_type = 5

    return curve_type